# Code handout - Assignment 5 - Ball and Beam

In this task, you will be asked to fill in the definitions and expressions necessary for the simulation of the "ball on beam" to run to completion.
If everything works out, there should be an animation of the externally driven beam joint that shows the behavior of the system.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

import sympy as sm
from sympy import vector
from IPython.display import display_latex

## Symbolic derivations

First, we define the symbols that will be necessary for the computations below.
These are:

- System parameters:
    - $J$, `J`: rotational inertia of the joint
    - $M$, `M`: mass of the ball
    - $R$, `R`: radius of the ball
    - $g$, `g`: gravitational acceleration (fun fact: locally in Trondheim, this is 9.82, not the usual 9.81)
- State:
    - $x$, `x`: displacement from the center of the beam
    - $\theta$, `theta`: angle of the rotation of the beam relative the the interial frame
    - $\dot{x}$, `x_dot`: time derivative of displacement
    - $\dot{\theta}$, `theta_dot`: time derivative of angle
- External torque
    - $T_o$, `torque`: external torque acting on the beam joint

In [2]:
# Defining the necessary symbols
J, M, R, g, torque = sm.symbols("J M R g T_o")
x, theta = sm.symbols("x theta")
x_dot, theta_dot = sm.symbols("xdot thetadot")

### Derivations and expressions

This is the section where you will have to fill in the correct expressions.
Specifically for:

- $q$, `q`: generalized coordinates of the system
- $\dot{q}$, `q_dot`: time derivative of the generalized coordinates
- $p$, `p`: position of ball center
- $T$, `T`: total kinetic energy, involving:
    - `kinetic_energy_beam`: 
    - `linear_energy_ball`: 
    - `rotational_energy_ball`: kinetic energy from the fact that the ball rotates
    - $I$, `I`: inertia of the rotating ball
    - $\omega$, `omega`: angular velocity of the ball
- $V$, `V`: potential energy of the system
- $Q$, `Q`: generalized forces

Hint: use `display_latex(<expr>)` to print the supplied expression, this can be handy for checking that what you've written is actually what you wanted to write!

In [3]:
placeholder = sm.symbols("a")

In [4]:
q = placeholder      # TODO
q_dot = placeholder  # TODO

In [5]:
p = placeholder      # TODO
p_dot = placeholder  # TODO

kinetic_energy_beam = placeholder  # TODO
linear_energy_ball = placeholder   # TODO

I = placeholder                       # TODO
omega = placeholder                   # TODO
rotational_energy_ball = placeholder  # TODO

In [6]:
T = placeholder  # TODO
V = placeholder  # TODO
Q = placeholder  # TODO
Lagrangian = placeholder  # TODO

### Lagrangian magic

Now, we let SymPy do what it needs to give us the expressions we need.
If the definitions you have made above are correct, you should be able to uncomment the computations and it should *just work*.
Good luck.

In [7]:
#Lagrangian_q = sm.simplify(sm.Matrix.jacobian(Lagrangian, q))
#Lagrangian_q_qdot = sm.simplify(sm.Matrix.jacobian(Lagrangian_q, q_dot))
#Lagrangian_qdot = sm.simplify(sm.Matrix.jacobian(Lagrangian, q_dot))
#Lagrangian_qdot_qdot = sm.simplify(sm.Matrix.jacobian(Lagrangian_qdot, q_dot))

In [8]:
#W = Lagrangian_qdot_qdot
#RHS = Q + sm.simplify(Lagrangian_q.T - Lagrangian_q_qdot @ q_dot)

### Lambdification

Converts the symbolic expressions to callable functions accelerated with NumPy.
We let the state vector be $[x, \theta, \dot{x}, \dot{\theta}]^T$
and the parameter vector be $[J, M, R, g]^T$.
Additionally, the $W$ and `RHS`-functions require the external torque $T$.

This corresponds to the routine mentioned in task 2e that exports functions for ball position and ODE matrices.
(uncomment the expressions here as well!)

In [9]:
state = sm.Matrix([q, q_dot])
param = sm.Matrix([J, M, R, g])

#ball_position = sm.lambdify((state, param), p, modules="numpy")
#get_W = sm.lambdify((state, param, torque), W, modules="numpy")
#get_RHS = sm.lambdify((state, param, torque), RHS, modules="numpy")

### Simulation

As usual, in order to use `scipy.integrate.solve_ivp`, we need to supply a function $\dot{q} = f(t, q)$, which we will wrap in a scope containing the necessary parameters.
The precise implementation of $f(t, q)$ is up to you!

In [10]:
def make_ball_and_beam_dynamics(param):
    def ball_and_beam_dynamics(time, state):
        # TODO
        pass
    return ball_and_beam_dynamics

Write the correct expressions for the state and parameters of the system and switch the placeholder `ys`/`ts` with the computed results.
Note that:

- `ys` is the state vector of the system for each computed time
- `ts` is the timestamp corresponding to each comptued state vector

In [11]:
from scipy.integrate import solve_ivp

# Parameters and initial states
time_final  = 15
parameters = None  # TODO
state = None       # TODO

ball_and_beam_dynamics = make_ball_and_beam_dynamics(parameters)
#res = solve_ivp(ball_and_beam_dynamics, (0, time_final), state)  # TODO

#ys = res.y  # TODO
#ts = res.t  # TODO

ys = np.zeros((4, 100), dtype=np.float32)
ts = np.zeros((1, 100), dtype=np.float32)

## Animation

In [12]:
import pythreejs as pj

## Initializing the scene
scene = pj.Scene()
camera = pj.PerspectiveCamera(position=[0, 0, 5], up=[0, 1, 0], aspect=1)
camera.lookAt([0, 0, 0])

## Setting up the skybox
folder = "ceiling_lights_cubemap"
texture_paths = [
    f"./{folder}/px.png",  # Positive X
    f"./{folder}/nx.png",  # Negative X
    f"./{folder}/py.png",  # Positive Y
    f"./{folder}/ny.png",  # Negative Y
    f"./{folder}/pz.png",  # Positive Z
    f"./{folder}/nz.png",  # Negative Z
]
box_sides = 500
geom = pj.BoxGeometry(width=box_sides, height=box_sides, depth=box_sides)
materials = [pj.MeshBasicMaterial(map = pj.ImageTexture(imageUri = path), side="BackSide") for path in texture_paths]
skybox = pj.Mesh(geom, materials)
scene.add(skybox)

## Setting up objects
ball_radius = 0.25
beam_length, beam_width, beam_height = 3, .5, .1

ball = pj.Mesh(
    pj.SphereGeometry(ball_radius, 32, 16), 
    pj.MeshStandardMaterial(color="blue"))
beam = pj.Mesh(
    pj.BoxGeometry(beam_length, beam_height, beam_width),
    pj.MeshStandardMaterial(color="red"))

## Position and rotation
ball_pos = np.zeros((3, ys.shape[1]), dtype=np.float32)
ball_pos[0, :] = ys[0, :]
ball_pos[1, :] = (ball_radius + 0.5 * beam_height) * np.ones((len(ts)))

from scipy.spatial.transform import Rotation
exaggeration_coefficient = 10
beam_rot = Rotation.from_euler("z", exaggeration_coefficient * ys[1, :], degrees=True).as_quat().astype(np.float32).T

beam.position = (0, 0, 0)
ball.position = tuple(ball_pos[:, 0])

## Collecting in a group for correct rotation animation
pivot = pj.Group()
pivot.add(beam)  # this becomes pivot.children[0]
pivot.add(ball)  # this becomes pivot.children[1]
pivot.quaternion = tuple(beam_rot[:, 0])
scene.add(pivot)

## Setting up the animation
ball_position_track = pj.VectorKeyframeTrack(name=".children[1].position", times = ts, values = ball_pos.T)
pivot_rotation_track = pj.QuaternionKeyframeTrack(name=".quaternion", times = ts, values = beam_rot.T)
pivot_clip = pj.AnimationClip(tracks = [ball_position_track, pivot_rotation_track])
pivot_action = pj.AnimationAction(pj.AnimationMixer(pivot), pivot_clip, pivot)

## Setting the scene
view_width, view_height = 800, 600
camera = pj.PerspectiveCamera(position=[0, 1, 4], aspect = view_width/view_height)
ambient_light = pj.AmbientLight(color="#ffffff", intensity=1.0)
key_light = pj.DirectionalLight(position=[0, 10, 0])
scene.add(ambient_light)
scene.add(key_light)

## Making the renderer
renderer = pj.Renderer(camera=camera, scene=scene, width=view_width, height=view_height)
controls = pj.OrbitControls(controlling = camera)
renderer.controls = [controls]

Now, we are finally ready for the animation!

In [13]:
display(renderer)
pivot_action

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, position=(0.0, 1.0, 4.0), projectionMatrix=(1.0, …

AnimationAction(clip=AnimationClip(tracks=(VectorKeyframeTrack(name='.children[1].position', times=array([[0.,…